In [1]:
import arviz as az
import matplotlib.pyplot as plt
import numpy as np
import pymc as pm
from pymc.math import switch

%load_ext watermark
%load_ext lab_black

# Psoriasis: two-sample problem with paired data

This is our first example of hypothesis testing.

Adapted from [Unit 6: psoriasis.odc](https://raw.githubusercontent.com/areding/6420-pymc/main/original_examples/Codes4Unit6/psoriasis.odc).

Associated lecture video: Unit 6 lesson 7

## Problem statement
Woo and McKenna (2003) investigated the effect of broadband ultraviolet B (UVB) therapy and topical calcipotriol cream used together on areas of psoriasis. One of the outcome variables is the Psoriasis Area and Severity Index (PASI), where a lower score is better.

The PASI scores for 20 subjects are measured at baseline and after 8 treatments.  Do these data provide sufficient evidence to indicate that the combination therapy reduces PASI scores?

Classical Analysis:
```
   d = baseline - after;
   n = length(d);
   dbar = mean(d);  dbar = 6.3550
   sdd = sqrt(var(d));  sdd = 4.9309
   tstat = dbar/(sdd/sqrt(n));  tstat = 5.7637

   Reject H_0 at the level alpha=0.05 since the p_value = 0.00000744 < 0.05

   95% CI is [4.0472, 8.6628]
   ```


---
See [Unit 6: Stress, Diet and Plasma Acids](https://areding.github.io/6420-pymc/unit6/Unit6-stressacids.html) to find out more about recreating the BUGS step function.

In [2]:
# fmt: off
baseline = np.array((5.9, 7.6, 12.8, 16.5, 6.1, 14.4, 6.6, 5.4, 9.6, 11.6, 
                     11.1, 15.6, 9.6, 15.2, 21.0, 5.9, 10.0, 12.2, 20.2, 
                     6.2))

after = np.array((5.2, 12.2, 4.6, 4.0, 0.4, 3.8, 1.2, 3.1, 3.5, 4.9, 11.1,
                  8.4, 5.8, 5, 6.4, 0.0, 2.7, 5.1, 4.8, 4.2))
# fmt: on

In [3]:
with pm.Model() as m:
    # priors
    mu = pm.Normal("mu", mu=0, sigma=316)
    prec = pm.Gamma("prec", alpha=0.001, beta=0.001)
    sigma = pm.Deterministic("sigma", 1 / pm.math.sqrt(prec))

    ph1 = pm.Deterministic("ph1", switch(mu >= 0, 1, 0))

    diff = pm.Normal("diff", mu=mu, sigma=sigma, observed=baseline - after)

    # start sampling
    trace = pm.sample(
        10000,
        chains=4,
        tune=500,
        cores=4,
        init="jitter+adapt_diag",
        random_seed=1,
        return_inferencedata=True,
    )

Auto-assigning NUTS sampler...
Initializing NUTS using jitter+adapt_diag...
/Users/aaron/mambaforge/envs/pymc_env2/lib/python3.11/site-packages/multipledispatch/dispatcher.py:27: AmbiguityWarning: 
Ambiguities exist in dispatched function _unify

The following signatures may result in ambiguous behavior:
	[object, ConstrainedVar, Mapping], [ConstrainedVar, object, Mapping]
	[object, ConstrainedVar, Mapping], [ConstrainedVar, Var, Mapping]
	[object, ConstrainedVar, Mapping], [ConstrainedVar, object, Mapping]
	[object, ConstrainedVar, Mapping], [ConstrainedVar, Var, Mapping]


Consider making the following additions:

@dispatch(ConstrainedVar, ConstrainedVar, Mapping)
def _unify(...)

@dispatch(ConstrainedVar, ConstrainedVar, Mapping)
def _unify(...)

@dispatch(ConstrainedVar, ConstrainedVar, Mapping)
def _unify(...)

@dispatch(ConstrainedVar, ConstrainedVar, Mapping)
def _unify(...)
  warn(warning_text(dispatcher.name, ambiguities), AmbiguityWarning)
Multiprocess sampling (4 chains in 4

Sampling 4 chains for 500 tune and 10_000 draw iterations (2_000 + 40_000 draws total) took 7 seconds.


In [4]:
az.summary(trace, hdi_prob=0.95, var_names=["~prec"])

/Users/aaron/mambaforge/envs/pymc_env2/lib/python3.11/site-packages/arviz/stats/diagnostics.py:584: RuntimeWarning: invalid value encountered in scalar divide
  (between_chain_variance / within_chain_variance + num_samples - 1) / (num_samples)


mean     sd  hdi_2.5%  hdi_97.5%  mcse_mean  mcse_sd  ess_bulk  \
mu     6.348  1.155     4.073      8.654      0.007    0.005   30917.0   
sigma  5.138  0.882     3.612      6.908      0.005    0.004   30678.0   
ph1    1.000  0.000     1.000      1.000      0.000    0.000   40000.0   

       ess_tail  r_hat  
mu      24883.0    1.0  
sigma   26152.0    1.0  
ph1     40000.0    NaN

Even though we're getting lots of warnings, our model agrees with the BUGS results:

|          | mean     | sd      | MC_error | val2.5pc | median | val97.5pc | start | sample |
|----------|----------|---------|----------|----------|--------|-----------|-------|--------|
| pH1      | 1        | 0       | 3.16E-13 | 1        | 1      | 1         | 1001  | 100000 |
| mu       | 6.352    | 1.169   | 0.003657 | 4.043    | 6.351  | 8.666     | 1001  | 100000 |
| sigma    | 5.142    | 0.8912  | 0.003126 | 3.74     | 5.026  | 7.203     | 1001  | 100000 |
| pval     | 4.20E-04 | 0.02049 | 6.04E-05 | 0        | 0      | 0         | 1001  | 100000 |

In [5]:
%watermark -n -u -v -iv -p aesara,aeppl

Last updated: Fri Feb 03 2023

Python implementation: CPython
Python version       : 3.11.0
IPython version      : 8.9.0

aesara: 2.8.10
aeppl : 0.1.1

pymc      : 5.0.1
numpy     : 1.24.1
arviz     : 0.14.0
matplotlib: 3.6.3

